In [9]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

import json

api_key = 'RGAPI-2bf73814-83af-4540-a2e0-46eee1c8271f'
watcher = LolWatcher(api_key=api_key)
my_region = 'na1'
me = watcher.summoner.by_name(my_region, 'yENatividad')

In [56]:
def json_data_mask(dic):
    keys_to_remove = []
    for k,v in dic.items():
        if type(v) is dict:
            keys_to_remove = keys_to_remove + [k]
    return keys_to_remove


def clean_timeframe(timeline, frame=5):
    players_list = list(timeline['info']['frames'][frame]['participantFrames'].values())
    keys_to_remove = json_data_mask(players_list[0])
    keys_to_remove += ['currentGold', 'goldPerSecond', 'participantId', 'totalDamageDone', 'totalDamageDoneToChampions', 'totalDamageTaken']
    for player in players_list:
        if 'damageStats' in player.keys():
            damage_stat = player['damageStats']
            player.update(damage_stat)
        for key in keys_to_remove:
            if key in player.keys():
                player.pop(key)
    return players_list

def add_creep_score(timeframe):
    if type(timeframe) is not list:
        raise TypeError("Input timeframe must be a list")
    elif type(timeframe[0]) is not dict:
        raise TypeError("Elements of inputted timeframe are not dicts.")
    
    for player in timeframe:
        if 'jungleMinionsKilled' in player.keys() and 'minionsKilled' in player.keys():
            player['creepScore'] = player['jungleMinionsKilled'] +player['minionsKilled']
            player.pop('jungleMinionsKilled')
            player.pop('minionsKilled')
        elif 'jungleMinionsKilled' not in player.keys() and 'minionsKilled' not in player.keys():
            if 'creepScore' not in player.keys():
                raise ValueError("Missing information to construct creep score for a player.")
        else:
            raise ValueError("Missing information to construct creep score for a player.")
    
    return timeframe

def process_timeframe(timeline, win, frame=10, matchid=None):
    cleaned = clean_timeframe(timeline, frame)
    cleaned = add_creep_score(cleaned)
    final_dict = {}
    for i in range(5):
        for key in list(cleaned[0].keys()):
            final_dict[key+'_'+str(i)] = cleaned[i][key] - cleaned[i+5][key]
    final_dict['matchId'] = matchid if matchid else 'UNKNOWN'
    final_dict['win'] = bool(win)
    return final_dict
    

## Match Timeline Summary

In [62]:
my_matches = watcher.match.matchlist_by_puuid(my_region, me['puuid'])
latest_matchid = my_matches[0]
latest_match = watcher.match.by_id(my_region, latest_matchid)
latest_timeline = watcher.match.timeline_by_match(my_region, latest_matchid)

In [63]:
label = latest_match['info']['participants'][0]['win']

summary = process_timeframe(latest_timeline, label, 10, latest_matchid)
pd.DataFrame([summary])

,level_0,timeEnemySpentControlled_0,totalGold_0,xp_0,magicDamageDone_0,magicDamageDoneToChampions_0,magicDamageTaken_0,physicalDamageDone_0,physicalDamageDoneToChampions_0,physicalDamageTaken_0,...,magicDamageTaken_4,physicalDamageDone_4,physicalDamageDoneToChampions_4,physicalDamageTaken_4,trueDamageDone_4,trueDamageDoneToChampions_4,trueDamageTaken_4,creepScore_4,matchId,win
0,0,88650,-910,-366,-1886,-759,759,-2869,79,-254,...,441,899,67,-50,1418,356,-90,5,NA1_4308240925,False
